In [ ]:
import sys
import os

In [ ]:
code_dir = os.environ.get('HALITE_PATH')

In [ ]:
if not code_dir:
    code_dir = '/'.join(os.getcwd().split('/')[:-1] + ['code'])

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.append(code_dir)

In [ ]:
from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import Board, ShipAction, ShipyardAction, Observation
from halite_env import HaliteEnv
from ship_state_wrapper import ShipStateWrapper
from shipyard_state_wrapper import ShipYardStateWrapper
from agent import Agent
from game_runner_v2 import GameRunner
from halite_agent import HaliteAgent

In [ ]:
from play import train_agent, play_games

In [ ]:
# https://www.kaggle.com/tiger37/reinforcement-learning-meets-halite

In [ ]:
# reward_results = train_agent(10, 400)

In [ ]:
a = [1,2,3,4,5,6]
a[len(a)//3:len(a)//3*2]

In [ ]:
results = play_games(3, 100)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
all_scores = results['all_scores']
ship_agent = results['ship_agent']
shipyard_agent = results['shipyard_agent']

In [ ]:
rewards = all_scores

In [ ]:
len(rewards)

In [ ]:
i = 0
for episode in rewards[-5:]:
    i += 1
    episode = np.array(episode)
    for j in range(episode.shape[1]):
        plt.subplot(3, 2, i)
        sns.lineplot(x=range(0, episode.shape[0]), y=episode[:, j])

## Playing Already Trained Agent

In [ ]:
environment = make('halite', {}, debug=True)

In [ ]:
trainer = environment.train([None, "random"])

## Re-Instantiating Wrappers Because They Hold State

In [ ]:
ship_state_wrapper = ShipStateWrapper(
    radius=4,
    max_frames=2,
    map_size=1
)
shipyard_state_wrapper = ShipYardStateWrapper(
    radius=4,
    max_frames=1,
    map_size=1
)

## Re-Instatiating Env

In [ ]:
halite_env = HaliteEnv(
    opponents=3,
    ship_state_wrapper=ship_state_wrapper,
    shipyard_state_wrapper=shipyard_state_wrapper,
    radius=4,
    environment=None,
    trainer=None
)

## Creating Agent

Note: In a submission file we should use an already trained agent and load from the Kaggle/Outputs directory.

In [ ]:
halite_agent = HaliteAgent(
    configuration=environment.configuration,
    halite_env=halite_env,
    ship_agent=ship_agent,
    shipyard_agent=shipyard_agent,
    training=False,
    verbose=False,
    ship_frame_stack_len=2
)

In [ ]:
from kaggle_environments.envs.halite.helpers import Board, Observation

## Create Agent Runner Function

In [ ]:
def halite_run_agent(observation, configuration):
    
    halite_agent.env.ship_state_wrapper.set_map_size(configuration['size'])
    halite_agent.env.shipyard_state_wrapper.set_map_size(configuration['size'])
    
    raw_observation = observation
    step_observation = Observation(observation)
    
    raw_observation, shipyard_simulated_step_memory = halite_agent.get_moves_for_all_shipyards(
        raw_observation=raw_observation,
        step_observation=step_observation,
        episode_number=0,
        step_number=0
    )

    raw_observation, ship_simulated_step_memory = halite_agent.get_moves_for_all_ships(
        raw_observation=raw_observation,
        step_observation=step_observation,
        episode_number=0,
        step_number=0
    )

    actions_for_step = halite_agent.actions_for_step
    return actions_for_step

In [ ]:
run_result = environment.run([halite_run_agent, "random", "random", "random"])

In [ ]:
len(run_result)

In [ ]:
run_result[-1]

In [ ]:
trainer = environment.train([None, "random"])

In [ ]:
state = trainer.reset()

In [ ]:
state.keys()

In [ ]:
halite_run_agent(state, environment.configuration)

In [ ]:
observation, game_reward, terminal = step_results

In [ ]:
%%writefile submission.py

def halite_run_agent(observation, configuration):
    
    halite_agent.env.ship_state_wrapper.set_map_size(configuration['size'])
    halite_agent.env.shipyard_state_wrapper.set_map_size(configuration['size'])
    
    raw_observation = observation
    step_observation = Observation(observation)
    
    raw_observation, shipyard_simulated_step_memory = halite_agent.get_moves_for_all_shipyards(
        raw_observation=raw_observation,
        step_observation=step_observation,
        episode_number=0,
        step_number=0
    )

    raw_observation, ship_simulated_step_memory = halite_agent.get_moves_for_all_ships(
        raw_observation=raw_observation,
        step_observation=step_observation,
        episode_number=0,
        step_number=0
    )

    actions_for_step = halite_agent.actions_for_step
    return actions_for_step
    